A metric approach to differentiate real data distribution from  synthetic data distribution generated by using the CycleGAN generator. A pretrained network is used as a feature extractor and the activations on the terminal node are used to model a gaussian distribution for each domain. The distance between the distributions is computed as the metric. 

The distance calculated by the metric is the wassertien distance (P-Q) between the real(P) and synthetic(Q) distributions. When the samples are very large the reflexive distance (P-P) should ideally be close to zero. In this notebook using limited samples this distance ( using method calculate_score(P, P) aka baseline score) is used as a placeholder baseline to compare with P-Q distance (calculate_score(P, Q) aka cross domain score). 

A couple of ways to test this metric:
(1) Check if the metric calculate_score(P, Q) decreases as the GAN training progresses and the synthetic distribution approaches the real distribution
(2) As the sample size increases the distance using samples from the same distribution should approch zero.



In [1]:
from torchvision.models import inception_v3,VGG,AlexNet
import torch
from torch import nn
import cv2
import numpy as np
import glob
import os
from scipy import linalg

In [2]:
# pre-requisites for repo - run if packages missing
!pip install -r requirements.txt
#Download model and dataset - ideally should use provided scripts from repo but running into cert. errors. USe alt. in next cell
#!bash ./scripts/download_cyclegan_model.sh horse2zebra
#!bash ./datasets/download_cyclegan_dataset.sh horse2zebra



    100% |████████████████████████████████| 686kB 25.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for visdom ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Running setup.py bdist_wheel for torchfile ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
#Alterative to download pre-trained model and dataset
!mkdir -p ./checkpoints/horse2zebra_pretrained
!wget http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/horse2zebra.pth -O ./checkpoints/horse2zebra_pretrained/latest_net_G.pth

!sudo yum update -y  ca-certificates
!wget https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip -O ./datasets/horse2zebra.zip
!mkdir ./datasets/horse2zebra/
!unzip ./datasets/horse2zebra.zip -d ./datasets/
!rm ./datasets/horse2zebra.zip

for details.

--2020-06-19 18:08:11--  http://efrosgans.eecs.berkeley.edu/cyclegan/pretrained_models/horse2zebra.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.189.73
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.189.73|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45575747 (43M)
Saving to: ‘./checkpoints/horse2zebra_pretrained/latest_net_G.pth’

./checkpoints/horse 100%[===================>]  43.46M  19.4MB/s    in 2.2s    

2020-06-19 18:08:14 (19.4 MB/s) - ‘./checkpoints/horse2zebra_pretrained/latest_net_G.pth’ saved [45575747/45575747]

for details.

--2020-06-19 18:08:14--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.189.73
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.189.73|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaitin

In [4]:
#Generate synthetic data using pre-trained generator from CycleGAN repo
!python test.py --dataroot datasets/horse2zebra/trainA --name horse2zebra_pretrained --model test --no_dropout

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: datasets/horse2zebra/trainA   	[default: None]
             dataset_mode: single                        
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [5]:
#Class to create activations from input images using pytorch module
class EvaluateCycleGAN(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_cnn = inception_v3(pretrained=True).eval()
        
        self.pretrained_cnn.Mixed_7c.register_forward_hook(self.fwd_hook)
        
        
    def fwd_hook(self, module,input, output):
        #print('Inside ' + self.__class__.__name__ + ' forward')
        self.mixed_7c_output = output
        
    def forward(self, x):
        self.pretrained_cnn(x)
        last_layer_avg_act = nn.functional.adaptive_avg_pool2d(self.mixed_7c_output,(1,1)).view(x.shape[0],2048)
        return last_layer_avg_act
        

In [6]:
#Resize image
def process_image(path):
    im = cv2.imread(path)
    im = im.astype(np.float32) / 255
    im = cv2.resize(im, (299, 299))
    im = np.rollaxis(im, axis=2)
    return im

#Extract activations on set of images
def extract_activations(pretrained_cnn,image_set):
    activations = []
    
    for im in image_set:
        activations.append(pretrained_cnn(im.unsqueeze(0).cuda()).detach().cpu())
    activations = np.stack(activations,axis=0)
    return activations
        

#Calcuate distance between the source and target distributions
def calculate_score(images1_path,images2_path):
    images1 = []
    images2 = []
    pretrained_cnn = EvaluateCycleGAN().cuda()
    for im1,im2 in zip(glob.glob(images1_path),glob.glob(images2_path)):
        image1 = process_image(im1)
        image2 = process_image(im2)
        images1.append(image1)
        images2.append(image2)

    images1 = torch.from_numpy(np.stack(images1,axis=0))
    images2 = torch.from_numpy(np.stack(images2,axis=0))
    activations1 = np.squeeze(extract_activations(pretrained_cnn,images1))
    activations2 = np.squeeze(extract_activations(pretrained_cnn,images2))
    mu1 = np.mean(activations1,axis=0)
    mu2 = np.mean(activations2,axis=0)

    sigma1 = np.cov(activations1, rowvar=False)
    sigma2 = np.cov(activations2, rowvar=False)


    sum_sqrd_diff = np.sum(np.power((mu1 - mu2),2.0))

    meanofcovariance = linalg.sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(meanofcovariance):
        meanofcovariance = meanofcovariance.real
    eval_metric = sum_sqrd_diff + np.trace(sigma1 + sigma2 - 2.0 * meanofcovariance)
    return eval_metric


Next cells validate the metric on real and synthetic datasets. Note that the number of samples for testing need to be much larger than what was used below.

In [7]:
#Get baseline score when comparing distributions from same domain
images1_path = 'datasets/horse2zebra/trainA/*'
images2_path = 'datasets/horse2zebra/testA/*'
baseline_score1 = calculate_score(images1_path,images2_path)
print(baseline_score1)



118.337727101751


In [8]:
#Get the cross domain score when comparing distributions of the different domains
images1_path = 'datasets/horse2zebra/trainB/*'
images2_path = 'datasets/horse2zebra/testA/*'
cross_domain_score1 = calculate_score(images1_path,images2_path)
images1_path = 'datasets/horse2zebra/trainA/*'
images2_path = 'datasets/horse2zebra/testB/*'
cross_domain_score2 = calculate_score(images1_path,images2_path)
print(cross_domain_score1)
print(cross_domain_score2)

264.70388711809017
259.46644586806303


In [9]:
#Get the score for synthetic data and compare to real from the same domain, should be closer to baseline score if distributions are similar.
images1_path = 'results/horse2zebra_pretrained/test_latest/images/*fake*'
images2_path = 'datasets/horse2zebra/testB/*'
real_vs_synthetic_score1 = calculate_score(images1_path,images2_path)
print(real_vs_synthetic_score1)

139.6324603943376
